In [17]:
import pandas as pd
import numpy as np
import os

os.getcwd()

data=pd.read_csv('compustat_annual_2000_2017_with link information.csv')


/Users/geshuhao/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2,17,33,37,955,956,957,962,976,977,982) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
missing_percentage=data.isnull().sum()/data.shape[0]
data1=data.loc[:,missing_percentage<0.7]
data2=data1._get_numeric_data()
data3=data2.fillna(data2.median())
print(data3.shape)
print(data2.head(5))

(108683, 370)
   GVKEY  LPERMNO  LPERMCO    LINKDT  datadate   fyear  ajex  ajp  currtr  \
0   1004    54594    20000  19720424  20000531  1999.0   1.0  1.0     1.0   
1   1004    54594    20000  19720424  20010531  2000.0   1.0  1.0     1.0   
2   1004    54594    20000  19720424  20020531  2001.0   1.0  1.0     1.0   
3   1004    54594    20000  19720424  20030531  2002.0   1.0  1.0     1.0   
4   1004    54594    20000  19720424  20040531  2003.0   1.0  1.0     1.0   

   fyr     ...      gsector     gsubind     naics  priusa   sic  spcindcd  \
0  5.0     ...         20.0  20101010.0  423860.0       1  5080     110.0   
1  5.0     ...         20.0  20101010.0  423860.0       1  5080     110.0   
2  5.0     ...         20.0  20101010.0  423860.0       1  5080     110.0   
3  5.0     ...         20.0  20101010.0  423860.0       1  5080     110.0   
4  5.0     ...         20.0  20101010.0  423860.0       1  5080     110.0   

   spcseccd  stko  dldte     ipodate  
0     925.0   0.0    

In [27]:
msk = np.random.rand(len(data)) < 0.8
train = data3[msk]
test = data3[~msk]

(108683,)


In [49]:
y= train.loc[:,'oiadp']

In [46]:
X=train.loc[:,train.columns!='oiadp']

In [47]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [ ]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):

    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
            best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

result = stepwise_selection(X, y)

print('resulting features:')
print(result)


Add  txs                            with p-value 0.0
Add  nopi                           with p-value 0.0
Add  txndba                         with p-value 0.0
Add  ceq                            with p-value 0.0
Add  txdc                           with p-value 0.0
Add  ibadj                          with p-value 0.0
Add  ao                             with p-value 0.0
Add  ivncf                          with p-value 0.0
Add  prstkc                         with p-value 0.0
Add  dd                             with p-value 0.0
Add  csho                           with p-value 0.0
Add  ebit                           with p-value 0.0
Drop prstkc                         with p-value 0.961321
